# Images

1. Learn how to break scenes into image chips
2. Test making predictions on image chips using AutoML
3. Learn how to make predictions on image chips using ML Engine

Reference: https://www.kaggle.com/byrachonok/keras-for-search-ships-in-satellite-image

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import libtiff

In [7]:
import PIL
PIL.__version__

'5.3.0'

In [8]:
from PIL import Image, ImageDraw

Scenes available from Kaggle dataset

In [9]:
os.listdir('../data/scenes/')

['lb_4.png',
 'lb_2.png',
 'lb_3.png',
 'lb_1.png',
 'sfbay_4.png',
 'sfbay_1.png',
 'sfbay_3.png',
 'sfbay_2.png',
 '20150718_184300_090b_3B_Visual.tiff']

Scenes downloaded from Planet API using the scene_ids in image chip training set

In [10]:
os.listdir('../data/tmp/')[:5]

['20161006_002711_1_0c1b_3B_Visual.tif',
 '20150830_000650_0b07_3B_Visual.tif',
 '20161011_185736_0c72_3B_Visual.tif',
 '20161018_193329_0c45_3B_Visual.tif',
 '20160710_182139_0c78_3B_Visual.tif']

### Working with Images

In [11]:
libtiff.tiff_file.tif_lzw.decode

<function tif_lzw.decode>

In [12]:
image = Image.open('../data/scenes/20150718_184300_090b_3B_Visual.tiff')
pix = image.load()  # gives pixel access

In [ ]:
image

In [14]:
pix

### Image size values

In [15]:
n_spectrum = 3  # aka. depth
width = image.size[0]
height = image.size[1]

In [16]:
width, height

(6424, 5085)

In [17]:
picture_vector = []
for chanel in range(n_spectrum):
    for y in range(height):
        for x in range(width):
            picture_vector.append(pix[x, y][chanel])


In [18]:
# Each coordinate actually has 4 values, does that make it a 4 spectrum img?
pix[102, 59]

(0, 0, 0, 0)

In [19]:
picture_vector = np.array(picture_vector).astype('uint8')
picture_tensor = picture_vector.reshape([n_spectrum, height, width]).transpose(1, 2, 0)

In [ ]:
plt.figure(1, figsize = (15, 30))

plt.subplot(3, 1, 1)
plt.imshow(picture_tensor)

plt.show()


In [21]:
picture_tensor = picture_tensor.transpose(2, 0, 1)

## Search on the Image

In [23]:
def cutting(x, y):
    '''Appears to copy an 3 x 80 x 80 section of the image using the x, y coords
    as the top left origin'''

    area_study = np.arange(3*80*80).reshape(3, 80, 80)
    for i in range(80):
        for j in range(80):
            area_study[0][i][j] = picture_tensor[0][y+i][x+j]
            area_study[1][i][j] = picture_tensor[1][y+i][x+j]
            area_study[2][i][j] = picture_tensor[2][y+i][x+j]
    area_study = area_study.reshape([-1, 3, 80, 80])
    area_study = area_study.transpose([0,2,3,1])
    area_study = area_study / 255
    sys.stdout.write('\rX:{0} Y:{1}  '.format(x, y))
    return area_study

In [24]:
def not_near(x, y, s, coordinates):
    '''What does this one do?
    
    That depends on what these input values are. I can guess that x and y will be of
    the same type/semantics as the cutting function, but don't know what s or 
    coordinates are yet.
    '''

    result = True
    for e in coordinates:
        if x+s > e[0][0] and x-s < e[0][0] and y+s > e[0][1] and y-s < e[0][1]:
            result = False
    return result

In [25]:
def show_ship(x, y, acc, thickness=5):
    '''Huh?'''
    for i in range(80):
        for ch in range(3):
            for th in range(thickness):
                picture_tensor[ch][y+i][x-th] = -1

    for i in range(80):
        for ch in range(3):
            for th in range(thickness):
                picture_tensor[ch][y+i][x+th+80] = -1
        
    for i in range(80):
        for ch in range(3):
            for th in range(thickness):
                picture_tensor[ch][y-th][x+i] = -1
        
    for i in range(80):
        for ch in range(3):
            for th in range(thickness):
                picture_tensor[ch][y+th+80][x+i] = -1

In [ ]:
# DONT RUN
step = 10; coordinates = []
for y in range(int((height-(80-step))/step)):
    for x in range(int((width-(80-step))/step) ):
        area = cutting(x*step, y*step)

        # MAKE PREDICTION
        
#         result = model.predict(area)
#         if result[0][1] > 0.90 and not_near(x*step,y*step, 88, coordinates):
#             coordinates.append([[x*step, y*step], result])
#             print(result)
#             plt.imshow(area[0])
#             plt.show()

In [ ]:
step = 10; coordinates = []
for y in range(int((height-(80-step))/step)):
    for x in range(int((width-(80-step))/step) ):
        area = cutting(x*step, y*step)
        plt.imshow(area[0])
        plt.show
        # break
        
        # MAKE PREDICTION
        
#         result = model.predict(area)
#         if result[0][1] > 0.90 and not_near(x*step,y*step, 88, coordinates):
#             coordinates.append([[x*step, y*step], result])
#             print(result)
#             plt.imshow(area[0])
#             plt.show()

X:5680 Y:1020  

KeyboardInterrupt: 